<a href="https://colab.research.google.com/github/DevBhuyan/Image-classification/blob/master/cats%20vs%20dogs%20overfitted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
tf.__version__

'2.3.0'

In [3]:
class mycallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = {}):
        if logs.get('accuracy') >= 0.95:
            print('Reached 95% accuracy, canceling training')
            self.model.stop_training = True

In [4]:
callbacks = mycallback()

In [5]:
import os
import zipfile

In [6]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O /tmp/cats_and_dogs_filtered.zip

--2020-10-12 20:27:26--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.213.128, 173.194.217.128, 172.217.204.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.213.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   125MB/s    in 0.5s    

2020-10-12 20:27:26 (125 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [12]:
zfile = '/tmp/cats_and_dogs_filtered.zip'
ref = zipfile.ZipFile(zfile, 'r')
ref.extractall('/images')
ref.close()

In [13]:
base_dir = '/images/cats_and_dogs_filtered'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [14]:
train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

print(train_cat_fnames)
print('\n\n\n\n\n\n\n\n\n\n\n\n\n')
print(train_dog_fnames)

print('total cat training images :', len(train_cat_fnames))
print('total dog training images :', len(train_dog_fnames))

print('total cat validation images :', len(os.listdir(validation_cats_dir)))
print('total dog validation images :', len(os.listdir(validation_dogs_dir)))

['cat.92.jpg', 'cat.250.jpg', 'cat.100.jpg', 'cat.317.jpg', 'cat.200.jpg', 'cat.242.jpg', 'cat.869.jpg', 'cat.786.jpg', 'cat.331.jpg', 'cat.683.jpg', 'cat.909.jpg', 'cat.581.jpg', 'cat.924.jpg', 'cat.687.jpg', 'cat.82.jpg', 'cat.816.jpg', 'cat.983.jpg', 'cat.708.jpg', 'cat.706.jpg', 'cat.905.jpg', 'cat.972.jpg', 'cat.631.jpg', 'cat.105.jpg', 'cat.473.jpg', 'cat.15.jpg', 'cat.793.jpg', 'cat.67.jpg', 'cat.371.jpg', 'cat.670.jpg', 'cat.906.jpg', 'cat.490.jpg', 'cat.185.jpg', 'cat.52.jpg', 'cat.852.jpg', 'cat.158.jpg', 'cat.125.jpg', 'cat.290.jpg', 'cat.856.jpg', 'cat.187.jpg', 'cat.132.jpg', 'cat.837.jpg', 'cat.891.jpg', 'cat.520.jpg', 'cat.277.jpg', 'cat.532.jpg', 'cat.283.jpg', 'cat.13.jpg', 'cat.761.jpg', 'cat.78.jpg', 'cat.562.jpg', 'cat.370.jpg', 'cat.965.jpg', 'cat.577.jpg', 'cat.361.jpg', 'cat.211.jpg', 'cat.634.jpg', 'cat.238.jpg', 'cat.926.jpg', 'cat.427.jpg', 'cat.46.jpg', 'cat.588.jpg', 'cat.453.jpg', 'cat.727.jpg', 'cat.180.jpg', 'cat.359.jpg', 'cat.288.jpg', 'cat.156.jpg', 'c

In [24]:
model = tf.keras.Sequential([
    keras.layers.Conv2D(16, (3, 3), activation = tf.nn.relu, input_shape = (150, 150, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(32, (3, 3), activation = tf.nn.relu),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation = tf.nn.relu),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation = tf.nn.relu),
    keras.layers.Dense(1, activation = tf.nn.sigmoid)
])

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 15, 15, 32)       

In [26]:
model.compile(optimizer = keras.optimizers.RMSprop(lr = 0.001),
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

In [22]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255)
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255)

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                   batch_size = 20,
                                                   class_mode = 'binary',
                                                   target_size = (150, 150))

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                       batch_size = 20,
                                                       class_mode = 'binary',
                                                       target_size = (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [27]:
history = model.fit(train_generator,
                   validation_data = validation_generator,
                   steps_per_epoch = 100,
                   epochs = 15,
                   validation_steps = 50,
                   verbose = 2,
                   callbacks = [callbacks])

Epoch 1/15
100/100 - 55s - loss: 0.6931 - accuracy: 0.5250 - val_loss: 0.6611 - val_accuracy: 0.5920
Epoch 2/15
100/100 - 54s - loss: 0.6602 - accuracy: 0.6210 - val_loss: 0.6100 - val_accuracy: 0.6640
Epoch 3/15
100/100 - 54s - loss: 0.6168 - accuracy: 0.6715 - val_loss: 0.6507 - val_accuracy: 0.6470
Epoch 4/15


KeyboardInterrupt: ignored